In [ ]:
%pip install face_recognition efficientnet tensorflow opencv-python-headless gradio kaggle 


  Using cached kaggle-1.7.4.5-py3-none-any.whl.metadata (16 kB)
  Using cached bleach-6.2.0-py3-none-any.whl.metadata (30 kB)
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached kaggle-1.7.4.5-py3-none-any.whl (181 kB)
Using cached bleach-6.2.0-py3-none-any.whl (163 kB)
Using cached python_slugify-8.0.4-py2.py3-none-any.whl (10 kB)
Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [kaggle]2m4/5 [kaggle]
Note: you may need to restart the kernel to use updated packages.


In [2]:

# dataset link --   https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d manjilkarki/deepfake-and-real-images
!unzip -q deepfake-and-real-images.zip -d dataset


Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
 99% 1.67G/1.68G [00:23<00:00, 319MB/s]
100% 1.68G/1.68G [00:23<00:00, 77.8MB/s]


## Import Dependencies

In [ ]:
import os
import cv2
import numpy as np
import random
import gradio as gr
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from pathlib import Path
from PIL import Image 
from tensorflow.keras.models import load_model


/home/uppercase/Workspace/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-15 01:48:01.936114: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 01:48:01.944020: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-15 01:48:02.224791: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFl

In [2]:
print("GPU available:", tf.config.list_physical_devices('GPU'))


GPU available: []


E0000 00:00:1760472098.327696    3226 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1760472098.411566    3226 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
def load_images(folder, label, size=(244, 244)):
    data = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, size)
            data.append((img, label))
    return data

In [4]:
base_path = "/home/uppercase/Workspace/Lab Projects/ML/deepfake_detection"

train_real = f"{base_path}/Dataset/Train/Real"
train_fake = f"{base_path}/Dataset/Train/Fake"

test_real = f"{base_path}/Dataset/Test/Real"
test_fake = f"{base_path}/Dataset/Test/Fake"

val_real = f"{base_path}/Dataset/Validation/Real"
val_fake = f"{base_path}/Dataset/Validation/Fake"


In [5]:
train_real_data =  load_images(train_real, 0)

In [2]:
train_fake_data =  load_images(train_fake, 1)

NameError: name 'load_images' is not defined

In [ ]:
train_data = train_real_data + train_fake_data

In [ ]:
test_real_data =  load_images(test_real, 0)
test_fake_data =  load_images(test_fake, 1)
test_data = test_real_data + test_fake_data

In [ ]:
val_real_data =  load_images(val_real, 0)
val_fake_data =  load_images(val_fake, 1)
val_data = val_real_data + val_fake_data

In [ ]:

# Convert to NumPy with reduced memory usage
X_train = np.array([item[0] for item in train_data], dtype=np.float32) / 255.0
y_train = np.array([item[1] for item in train_data])
y_train = to_categorical(y, num_classes=2)

X_test = np.array([item[0] for item in test_data], dtype=np.float32) / 255.0
y_test = np.array([item[1] for item in test_data])
y_test = to_categorical(y_test, num_classes=2)

X_val = np.array([item[0] for item in val_data], dtype=np.float32) / 255.0
y_val = np.array([item[1] for item in val_data])
y_val = to_categorical(y_val, num_classes=2)


# Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)


In [ ]:
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(244, 244, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    datagen.flow(X_train, y_train, batch_size=16),
    epochs=5,
    validation_data=(X_test, y_test)
)
model.save("deepfake_detector_efficientnetb4.h5")

In [ ]:
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Step 4: Recompile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Continue training (fine-tuning)
model.fit(
    datagen.flow(X_train, y_train, batch_size=16),
    epochs=5,
    validation_data=(X_test, y_test)
)

# Step 6: Save the fine-tuned model
model.save("deepfake_detector_efficientnetb4_finetuned.h5")

In [ ]:

known_faces = []
known_names = []

dataset_path = Path(r"D:\new_project\face_dataset")

# Accept common image file extensions
valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}

for img_path in dataset_path.rglob("*"):
    if img_path.suffix.lower() not in valid_extensions:
        continue

    try:
        image = face_recognition.load_image_file(str(img_path))
        encodings = face_recognition.face_encodings(image)

        if encodings:
            known_faces.append(encodings[0])
            name = img_path.parent.name  # Use folder name as label
            known_names.append(name)
        else:
            print(f"No face found in: {img_path.name}")

    except Exception as e:
        print(f"Error processing {img_path.name}: {e}")


In [ ]:

# Load your trained model before prediction
model = load_model("deepfake_detector_efficientnetb4_finetuned.h5")

def predict_and_recognize(image, is_webcam=False):
    try:
        # Deepfake Detection (always performed)
        # Webcam input is resized and processed for deepfake detection
        image_resized = cv2.resize(image, (244, 244)) / 255.0
        input_tensor = np.expand_dims(image_resized, axis=0)

        pred = model.predict(input_tensor)[0]
        predicted_class = np.argmax(pred)
        confidence = round(float(np.max(pred)) * 100, 2)

        is_real = predicted_class == 0 and confidence >= 50.0  # Adjusted threshold slightly
        result = "Real" if is_real else "Fake"
        full_result = f"{result} Face ({confidence}% Confidence)"

        # Removed Face recognition part due to CUDA error with face_recognition library

        return full_result # Return only deepfake detection result

    except Exception as e:
        return f"Error during prediction: {str(e)}"

In [ ]:
# Load your trained model before prediction (re-added here in case it was removed elsewhere)
# model = load_model("deepfake_detector_efficientnetb4_finetuned.h5") # Assuming model is loaded in a previous cell

interface = gr.Interface(fn=predict_and_recognize,
                         inputs=gr.Image(type="numpy", label="Upload or Capture Face Image"),
                         outputs="text",
                         title="Deepfake Detection", # Updated title
                         description="Upload a face image or use your webcam to detect if it's a deepfake.") # Updated description

interface.launch(share=True)